# 编程和测试要求
## 3. 拓展功能

> 考虑到向实用性扩展，加密算法的数据输入可以是ASII编码字符串(分组为1 Byte)，对应地输出也可以是ACII字符串(很可能是乱码)。

我们这里选择对加密结果按照16进制的格式输出，避免输出乱码或者无法显示的系统控制符：

In [1]:
# 加载实现的加密算法类以及测试用类
from SDES.SDES import SDES
from unittest.mock import Mock

使用S-DES加密算法进行加密/解密处理...


### 3.1 ASCII码的处理

在后端的视图函数中根据用户的选择处理不同格式的输入：

In [2]:
# 用户明文处理函数
def plaintext_sent(request):
    if request.method == 'POST':
        # 接收用户输入
        key_input = request.POST['key']
        plaintext_input = request.POST['plaintext']
        type = request.POST['type'] 
        
        # 用户选择输入ASCII编码
        if (type == "ASCII"):
            sdes = SDES(key=key_input)
            out = ""
            for i in range(len(plaintext_input)):
                temp = bin(ord(plaintext_input[i])).replace("0b","").zfill(8)
                out_bin = sdes.encrypt(temp)
                out_hex = hex(int(out_bin, 2))[2:].zfill(2)
                out += (out_hex)
            data = {
                "ciphertext" : out
            }
            print("加密后的密文是: ",out)
            return data
        # 用户选择Bit编码
        elif(type == "Bit"):
            sdes = SDES(key=key_input)
            out_bin = sdes.encrypt(plaintext_input)
            data = {
                "ciphertext": out_bin
            }
            print("加密后的密文是: ",out_bin)
            return data


我们将模拟一个用户在前端生成的HttpRequest请求来测试:

In [3]:
# 创建一个模拟的 HttpRequest 对象
request_plain = Mock()

# 设置模拟对象的属性
request_plain.method = 'POST'
request_plain.POST = {
    'key': '0101010101',
    'plaintext': 'code',
    'type': 'ASCII',
}
# 测试
response = plaintext_sent(request_plain)

加密后的密文是:  24deae96


### 3.2 对密文的处理

在后端的视图函数中根据用户的选择处理不同格式的输入：

In [4]:
# 用户输入待转换的密文
def ciphertext_sent(request):
    if request.method == 'POST':
        # 获取用户输入
        key_input = request.POST['key']
        ciphertext_input = request.POST['plaintext']
        type = request.POST['type']
        # 用户选择输入ASCII编码
        if(type == "ASCII"):
            sdes = SDES(key=key_input)
            out = ""
            for i in range(len(ciphertext_input)//2):
                temp1 = bin(int(ciphertext_input[2*i], 16)).replace("0b", "").zfill(4)
                temp2 = bin(int(ciphertext_input[2*i+1], 16)).replace("0b", "").zfill(4)
                out_bin = sdes.decrypt(temp1+temp2)
                out_chr=chr(int(out_bin, 2))
                out += (out_chr)
            data = {
                "ciphertext": out
            }
            print("解密后的明文是:",out)
            return data
        
        # 用户选择输入Bit编码
        elif(type == "Bit"):
            sdes = SDES(key=key_input)
            out_bin = sdes.decrypt(ciphertext_input)
            data = {
                "ciphertext": out_bin
            }
            print("解密后的明文是:",out_bin)
            return data

In [5]:
# 创建一个模拟的 HttpRequest 对象并输入
request_cipher = Mock()

# 设置模拟对象的属性
request_cipher.method = 'POST'
request_cipher.POST = {
    'key': '0101010101',
    'plaintext': '24deae96',
    'type': 'ASCII',
}

#函数测试
response = ciphertext_sent(request_cipher)

解密后的明文是: code


经过一次模拟的用户请求测试，可以看出我们对ASCII码的处理无误。